In diesem Notebook wird die Pipeline aufgebaut.

In [1]:
%load_ext autoreload
%autoreload 2

import os
if os.getcwd() == '/home/jovyan/work': # jhub
    os.chdir("24FS_I4DS27/main/") 
    os.system("make reqs")
else: # local
    os.chdir("../")

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

from src.data.mri import MRIDataModule
from src.data.covidx import COVIDXDataModule
from src.utils.download import download_models
from src.utils.transform_perturbation import AddImagePerturbation
from src.utils.uap_helper import generate_adversarial_images_from_model_dataset, get_model
from src.utils.adv_training import pipeline, get_transform
from src.models.imageclassifier import ImageClassifier

from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.figsize"] = (16, 8)

torch.set_float32_matmul_precision('high')

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid v

In [3]:
ENTITY = "24FS_I4DS27"
PROJECT = "baselines"
NUM_WORKERS = 16

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [4]:
# Download models if not present
models = download_models(ENTITY, PROJECT)

In [5]:
modelname, dataset = "resnet50", "covidx_data"
print(f"\n---\nModel: {modelname} - Dataset: {dataset}")
print(f"Device: {device}")


lr = 1e-1
max_retries = 10
for attempt in range(max_retries):
    try:
        pipeline(
            modelname=modelname,
            dataset=dataset,
            n_robustifications=10,
            i=5,
            n=200,
            t=30,
            p=2,
            lambda_norm=0.001,
            r=0.5,
            eps=1e-6,
            lr_uap=lr,
            seed=42,
            num_workers=NUM_WORKERS,
            device=device,
            verbose=False,
        )
    # CUDA out of memory error
    except RuntimeError as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        if attempt >= max_retries - 1:
            print("All retries failed, lowering learning rate")
            lr /= 10**0.5


---
Model: resnet50 - Dataset: covidx_data
Device: cuda
Skipping robustification 0, since already exists
Skipping robustification 1, since already exists
Skipping robustification 2, since already exists
Skipping robustification 3, since already exists
Skipping robustification 4, since already exists
Skipping robustification 5, since already exists
Skipping robustification 6, since already exists
Skipping robustification 7, since already exists
Skipping robustification 8, since already exists
Skipping robustification 9, since already exists


KeyboardInterrupt: 